# Corpusanalyse UiTdatabank

We hebben de volgende bibliotheken nodig:

In [30]:
from pandas import read_excel, read_csv, DataFrame, Series, concat
from datetime import datetime
from codecs import open
from re import compile
from json import dumps
from datetime import datetime
from random import sample
from collections import Counter
from itertools import combinations

Dan lezen we lezen de gegevens in

In [2]:
df_podium = read_excel("ruwe data/podium.xlsx", sheetname='theaterdans1014')
df_bk = read_excel("ruwe data/beeldendekunsten.xlsx", sheetname='UitRapport')
df_muziek = read_excel("ruwe data/muziek.xlsx", sheetname='Int nat reg')
df_podium["Organisator"] = df_podium["Typologie organisator"]

## Samenbrengen en voorbereiden van de data

Voor de muziekgegevens moeten we nog controleren dat concerten eventuele herhalingen hebben, door in de speelmomenten kolom na te gaan wat de speelmomenten zijn. Dit is niet nodig bij de beeldende kunsten, aangezien daar de unit of analysis de tentoonstelling is, onafhankelijk van hoelang die tentoonstelling loopt. Bij podiumkunsten zijn de speelmomenten manueel gecheckt, ocharme simon.

In [3]:
datumregex = compile(r"\d\d/\d\d/\d\d")
df_muziek_expanded = df_muziek.copy()
for row in df_muziek_expanded.iterrows():
    speelmomenten = row[1]["Speelmomenten"]
    if str(speelmomenten) != "nan":    
        for speelmoment in datumregex.findall(speelmomenten):
            speelmoment_dt = datetime(int("20" + speelmoment.split("/")[2]), 
                                      int(speelmoment.split("/")[1]), 
                                      int(speelmoment.split("/")[0]))
            if speelmoment_dt != row[1]["Datum"]:
                df_muziek_expanded = df_muziek_expanded.append(
                    Series(
                        {"Discipline": row[1]["Discipline"],
                         "Subdiscipline": row[1]["Subdiscipline"],
                         "Tekst": row[1]["Tekst"],
                         "Datum": speelmoment_dt,
                         "Gemeente": row[1]["Gemeente"]
                        }, name=speelmoment_dt.isoformat() + " " + str(row[0])
                    )
                )

Voor de tentoonstellingen moeten we ook nog inperken op thema, en ook de permanente tentoonstellingen eruit zwieren.

In [4]:
subdisciplines = ["Beeldhouwkunst", 
                  "Fotografie", 
                  "Grafiek", 
                  "Installatiekunst", 
                  "Kunst en kunsteducatie", 
                  "Meerdere kunstvormen", 
                  "Schilderkunst"]
df_bk_filtered = df_bk[df_bk["Datum tot"] != datetime(1900, 1, 1)]
df_bk_filtered = df_bk_filtered[df_bk_filtered["Subdiscipline"].isin(subdisciplines)]
df_bk_filtered = df_bk_filtered[
    (df_bk_filtered["Datum van"].between(datetime(2014, 1, 1), datetime(2014, 12, 31))) |
    (df_bk_filtered["Datum tot"].between(datetime(2014, 1, 1), datetime(2014, 12, 31)))
]
df_bk_filtered["Datum"] = df_bk_filtered["Datum van"]
df_bk_filtered = df_bk_filtered.drop(["Datum van", "Datum tot"], axis=1)

Even kijken hoe dit er allemaal uitziet:

In [5]:
df_bk_filtered.head()

,Discipline,Tekst,Gemeente,Titel,Subdiscipline,Organisator,Datum
1,Tentoonstelling,Tentoonstelling keramiek van' de negen' afgest...,Hasselt,de ne9en,Meerdere kunstvormen,Stedelijke Academie voor Schone Kunsten Hasselt,2014-11-13
2,Tentoonstelling,Winterexpositie met werken van oa Panamaren...,Kemzeke (Stekene),Noli Me Tangere,Meerdere kunstvormen,Verbeke Foundation,2014-12-10
6,Tentoonstelling,"Project gebaseerd op technologie, instinct en ...",Brussel,Fragments #43-44,Installatiekunst,PointCulture Bruxelles,2013-12-17
8,Tentoonstelling,‘’Tentoonstelling – Tuin - Serres - Vrije Teke...,Gent,Tentoonstelling Modelstudies in Plantentuin UGent,Schilderkunst,NaN,2014-05-08
10,Tentoonstelling,In het kader van Kunst in het Gemeentehuis ste...,Vaalbeek (Oud-Heverlee),Kunst in het gemeentehuis: Lejon-art 70,Schilderkunst,Gemeente Oud-Heverlee,2014-09-24


In [6]:
df_muziek_expanded.head()

,Discipline,Tekst,Datum,Gemeente,Speelmomenten,Subdiscipline,Titel,Organisator
0,Concert,De aangrijpende film die internationaal hoge t...,2014-12-21 00:00:00,Brussel,zo 21/12/14 van 20:00 tot 22:30,Pop en rock,The Broken Circle Breakdown Bluegrass Band,Ancienne Belgique
1,Concert,Zelfs heel wat toppianisten deinzen ervoor ter...,2014-03-20 00:00:00,Antwerpen 18 (Antwerpen),do 20/03/14 van 20:00 tot 22:00,Thema onbepaald,National Philharmonic Of Russia Olv. Vladimir ...,deSingel
2,Concert,“Brilliant young piano trio.” (The Daily Teleg...,2014-10-23 00:00:00,Brussel,do 23/10/14 van 20:00 tot 22:30,Pop en rock,Gogo Penguin: Where Aphex Twin meets Jazz,Ancienne Belgique
3,Concert,"Het Salagon Quartett, waarvan niemand minder d...",2014-03-28 00:00:00,Brugge,vrij 28/03/14 om 20:00,Klassieke muziek,SALAGON QUARTETT / MOZART & BEETHOVEN,Concertgebouw Brugge
4,Concert,De Romeo’s hebben tal van nummer 1-hits op hun...,2014-10-31 00:00:00,Merksem (Antwerpen),vrij 31/10/14 om 20:00,Pop en rock,Feestival,NaN


In [7]:
df_podium.head()

,Tekst,Gemeente,Datum,Discipline,Subdiscipline,Titel,Typologie organisator,Organisator
0,Met Gil Grainlune.,Brussel,2010-01-12,podium,Theatervoorstelling,Raconte nous des histoires Mr Orts..,CA,CA
1,"Regie David Michels en Bernard Lefrancq, met R...",Brussel,2010-01-31,podium,Theatervoorstelling,La revue 2010,FRG,FRG
2,"Van M. de Ghelderode, bewerking José Géal en ...",Brussel,2010-01-02,podium,Theatervoorstelling,La Nativité et le massacre des innocents,LOT,LOT
3,"Van Jean-Michel Ribes, regie Eric De Staercke,...",Brussel,2010-01-03,podium,Theatervoorstelling,Théâtre sans animaux,FRG,FRG
4,"Van Georges Courteline, regie Valéry Massion, ...",Brussel,2010-01-09,podium,Theatervoorstelling,Faire le malin est le propre de tout imbécile,FRG,FRG


We gaan nu de de organisator omzetten naar een typering. We moeten eerst beginnen met die mapping van organisator naar typering op te bouwen op basis van de gegevens die Simon wist aan te leveren.

In [8]:
typering_podium = read_excel("extra gegevens/typologie_organisatoren.xlsx", sheetname='podium')
typering_muziek = read_excel("extra gegevens/typologie_organisatoren.xlsx", sheetname='muziek2')
typering_beeldende = read_excel("extra gegevens/typologie_organisatoren.xlsx", sheetname='beeldende1')

In [ ]:
onmapbaar = []

def map_organisator_naar_typologie(item, mapping):
    try:
        return mapping[mapping["key"].str.lower() == str(item).lower()]["value"].values[0]
    except IndexError:
        onmapbaar.append(item)

df_podium["typering"] = df_podium["Organisator"].apply(map_organisator_naar_typologie, args=(typering_podium,))
df_muziek_expanded["typering"] = df_muziek_expanded["Organisator"].apply(map_organisator_naar_typologie, args=(typering_muziek,))
df_bk_filtered["typering"] = df_bk_filtered["Organisator"].apply(map_organisator_naar_typologie, args=(typering_beeldende,))

Nu voegen we alles mooi samen, selecteren enkel de juiste kolommen, en gooien ook duplicaten op basis van datum, gemeente en tekst eruit. Bovendien hebben we de kolom met speelmomenten ook niet meer nodig.

In [9]:
df_podium.head()

,Tekst,Gemeente,Datum,Discipline,Subdiscipline,Titel,Typologie organisator,Organisator
0,Met Gil Grainlune.,Brussel,2010-01-12,podium,Theatervoorstelling,Raconte nous des histoires Mr Orts..,CA,CA
1,"Regie David Michels en Bernard Lefrancq, met R...",Brussel,2010-01-31,podium,Theatervoorstelling,La revue 2010,FRG,FRG
2,"Van M. de Ghelderode, bewerking José Géal en ...",Brussel,2010-01-02,podium,Theatervoorstelling,La Nativité et le massacre des innocents,LOT,LOT
3,"Van Jean-Michel Ribes, regie Eric De Staercke,...",Brussel,2010-01-03,podium,Theatervoorstelling,Théâtre sans animaux,FRG,FRG
4,"Van Georges Courteline, regie Valéry Massion, ...",Brussel,2010-01-09,podium,Theatervoorstelling,Faire le malin est le propre de tout imbécile,FRG,FRG


In [10]:
df_muziek_expanded_clean = df_muziek_expanded.drop("Speelmomenten", axis=1)
df_podium_clean = df_podium.drop("Typologie organisator", axis=1)
df = concat([df_podium_clean, df_bk_filtered, df_muziek_expanded_clean])
df = df[df["Datum"].between(datetime(2014, 1, 1), datetime(2014, 12, 31))]
df.drop_duplicates(subset=["Datum", "Titel", "Discipline", "Subdiscipline", "Gemeente", "Tekst"], inplace=True)
df.drop("Titel", axis=1, inplace=True)

We kunnen kort inspecteren hoe deze data eruitzien.

In [11]:
df.head()

,Datum,Discipline,Gemeente,Organisator,Subdiscipline,Tekst
747,2014-04-02 00:00:00,podium,Brussel,RE,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...
748,2014-04-03 00:00:00,podium,Brussel,RE,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...
749,2014-04-04 00:00:00,podium,Brussel,RE,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...
750,2014-04-05 00:00:00,podium,Brussel,RE,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...
752,2014-04-06 00:00:00,podium,Brussel,RE,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...


We zien dat elke lijn een event beschrijving bevat, de plaats waar een event plaatsvindt, en ook de datum. Merk op dat events die op meerdere dagen plaatsvinden een aparte lijn krijgen. We zullen hiermee rekening houden in de interpretatie van de resultaten.

Telling per discipline van het aantal unieke events:

In [12]:
df["Discipline"].value_counts()

Concert            20500
podium             19406
Tentoonstelling     3570
Name: Discipline, dtype: int64

En hoeveel daarvan hebben geen beschrijving?

In [30]:
print("podium", len(df[(df["Tekst"].str.strip() == "") & (df["Discipline"] == "podium")]["Tekst"]),
      "concert", len(df[(df["Tekst"].str.strip() == "") & (df["Discipline"] == "Concert")]["Tekst"]),
      "beeldend", len(df[(df["Tekst"].str.strip() == "") & (df["Discipline"] == "Tentoonstelling")]["Tekst"]))

podium 1521 concert 1842 beeldend 51


Zo, we kunnen deze dataset nu mooi uitschrijven naar een Excel bestand, zodat Simon nog enkele laatste correcties en aanvullingen kan doorvoeren.

In [14]:
df.to_csv("samengevoegde data/df.csv")

## Landsvermeldingen

Na de manuele correctie van Simon kunnen we nu aan de slag met een propere dataset, die we dan hier nu ook inlezen.

In [4]:
df = read_excel("samengevoegde data/df_manuele-correctie.xlsx")

We kunnen voor de volledigheid eventjes een overzicht maken van (organisatoren, discipline).

In [13]:
df.groupby(["typering", "Discipline"]).size()

typering                       Discipline     
Andere overheden               Concert            1595
                               Tentoonstelling      42
                               podium             2199
Lokaal (cultuur)beleid         Concert            3874
                               Tentoonstelling     776
                               podium             6228
Onderwijs                      Concert             377
                               Tentoonstelling      92
                               podium              304
Privaat                        Concert            3147
                               Tentoonstelling     458
                               podium             2843
Rest                           Concert             951
                               Tentoonstelling      69
                               podium              295
Sociaal-cultureel en amateur   Concert            2642
                               Tentoonstelling     240
                  

Voor onze analyse hebben we ook nood aan een lijst van namen van landen, coordinaten voor de plaatsnamen, en ook een manueel gemaakte mapping om de plaatsnamen in de UiTdatabank gegevens te normaliseren.

In [42]:
typering = read_csv("extra gegevens/mapping_udb-gemeente_fusie-gemeente.csv", delimiter=';')
coord = read_csv("extra gegevens/coordinaten.csv", delimiter=';')
landen = read_excel("extra gegevens/landen.xlsx", sheetname="uitgebreide lijst 2014")

Laten we even in detail deze tabellen bekijken. De landen:

In [20]:
landen.head()

,Continent (staatkundig),Land,Soort mention,Mention
0,Azië,Afghanistan,korte landnaam,Afghanistan
1,Azië,Afghanistan,inwoner,Afghaan
2,Azië,Afghanistan,inwoner,Afghanen
3,Azië,Afghanistan,adjectief,Afghaans
4,Azië,Afghanistan,adjectief,Afghaanse


Voor elk land weten we in welk (staatkundig) continent het ligt, en we hebben in de kolom 'Mention' verschillende manieren waarop dat land kan voorkomen in de tekst.

Bij typering zien we het volgende:

In [41]:
typering.tail()

,Gemeente Origineel,Postcode,Fusiegemeente,Stedelijkheid fusiegemeenten,Gemeente cluster,Province (English),Country (English),Provincie,Stedelijkheid
932,Kleine-Brogel (Peer),NaN,Peer,Niet-stedelijk,Peer,Limburg,Belgium,Limburg,Niet-stedelijk
933,Meilegem (Zwalm),NaN,Zwalm,Niet-stedelijk,Zwalm,East Flanders,Belgium,Oost-Vlaanderen,Niet-stedelijk
934,Beek (Bree),NaN,Bree,Niet-stedelijk,Bree,Limburg,Belgium,Limburg,Niet-stedelijk
935,Oetingen (Gooik),NaN,Gooik,Niet-stedelijk,Gooik,Flemish Brabant,Belgium,Vlaams-Brabant,Niet-stedelijk
936,Vaalbeek (Oud-Heverlee),NaN,Oud-Heverlee,Niet-stedelijk,Oud-Heverlee,Flemish Brabant,Belgium,Vlaams-Brabant,Niet-stedelijk


De kolom Gemeente Origineel is de naam van de gemeente in de uitdatabank gegevens, en we kunnen de naam in de kolom Fusiegemeente en Province (English) gebruiken om een genormaliseerd zicht te krijgen.

Tot slot hebben we nog de coordinaten:

In [22]:
coord.head()

,Fusiegemeente,adress,latitude,longitude
0,Aalst,"Aalst, Belgium","50,9378101","4,0409517"
1,Aalter,"Aalter, Belgium","51,087349","3,448371"
2,Aarschot,"Aarschot, Belgium","50,9859959","4,8365218"
3,Aartselaar,"Aartselaar, Belgium","51,1340539","4,3844742"
4,Affligem,"Affligem, Belgium","50,9084","4,11281"


Hiermee kunnen we voor iedere Fusiegemeente (zie vorige tabel) de latitude en longitude ophalen.

Nu gaan we voor ieder event in de UiTdatabankgegevens na welk land er vermeld wordt in de beschrijving van dat event. We houden ook al onmiddellijk bij wat de genormaliseerde naam is van de gemeente en de coordinaten van het centrum. Bovendien tellen we binnen het land ook nog de verschillende disciplines.

In [45]:
data = {}
niet_vlaams = ["Jodoigne", "Tournai", "Escanaffles", "Houffalize", "Haulchin", 
               "Braine l'Alleud", "Tourinnes-la-Grosse", "Liège", "Marchienne-au-Pont", "Eupen"]
count = 1
for row in df.iterrows():
    if count % 5000 == 0:
        print(count, "of", len(df.index))
    count += 1
    tekst = row[1]["Tekst"]
    gemeente = row[1]["Gemeente"]
    organisatie = row[1]["typering"]
    if str(gemeente) != "nan" and str(gemeente) not in niet_vlaams:
        for land in set(landen["Land"].values):
            regex = compile(r"\b(" + r"|".join(landen[landen["Land"] == land]["Mention"]) + r")\b")
            amount_of_mentions = len(regex.findall(str(tekst)))
            if amount_of_mentions > 0:
                try:
                    typeringlijn = typering[typering["Gemeente Origineel"] == gemeente]
                    fusiegemeente = typeringlijn["Fusiegemeente"].values[0]
                    provincie = typeringlijn["Province (English)"].values[0]
                    lat = coord[coord["Fusiegemeente"] == fusiegemeente]["latitude"].values[0]
                    lon = coord[coord["Fusiegemeente"] == fusiegemeente]["longitude"].values[0]
                    continent = landen[landen["Land"] == land]["Continent (staatkundig)"].values[0]
                    discipline = row[1]["Discipline"]
                    subdiscipline = row[1]["Subdiscipline"]
                    if fusiegemeente not in data:
                        data[fusiegemeente] = {"latitude": float(lat.replace(',', '.')),
                                               "longitude": float(lon.replace(',', '.')),
                                               "provincie": provincie,
                                               "vermeldingen": {}
                                               }
                    if organisatie not in data[fusiegemeente]["vermeldingen"]:
                        data[fusiegemeente]["vermeldingen"][organisatie] = {}
                    if continent not in data[fusiegemeente]["vermeldingen"][organisatie]:
                        data[fusiegemeente]["vermeldingen"][organisatie][continent] = {}
                    if land not in data[fusiegemeente]["vermeldingen"][organisatie][continent]:
                        data[fusiegemeente]["vermeldingen"][organisatie][continent][land] = {}
                    if discipline not in data[fusiegemeente]["vermeldingen"][organisatie][continent][land]:
                        data[fusiegemeente]["vermeldingen"][organisatie][continent][land][discipline] = {}
                    if subdiscipline not in data[fusiegemeente]["vermeldingen"][organisatie][continent][land][discipline] :
                        data[fusiegemeente]["vermeldingen"][organisatie][continent][land][discipline][subdiscipline] = 0
                    data[fusiegemeente]["vermeldingen"][organisatie][continent][land][discipline][subdiscipline] += amount_of_mentions
                except Exception as e:
                    print(gemeente, "geeft een probleem:", e)

5000 of 43450
10000 of 43450
15000 of 43450
20000 of 43450
Lessines geeft een probleem: index 0 is out of bounds for axis 0 with size 0
25000 of 43450
30000 of 43450
Charleroi geeft een probleem: index 0 is out of bounds for axis 0 with size 0
35000 of 43450
40000 of 43450
LiÍÎge geeft een probleem: index 0 is out of bounds for axis 0 with size 0
LiÍÎge geeft een probleem: index 0 is out of bounds for axis 0 with size 0
LiÍÎge geeft een probleem: index 0 is out of bounds for axis 0 with size 0
LiÍÎge geeft een probleem: index 0 is out of bounds for axis 0 with size 0
LiÍÎge geeft een probleem: index 0 is out of bounds for axis 0 with size 0
LiÍÎge geeft een probleem: index 0 is out of bounds for axis 0 with size 0


We kunnen al eens gauw de structuur van deze gegevens nakijken voor bijvoorbeeld de gemeente Lanaken:

In [50]:
print(dumps(data["Ruiselede"], indent=2))

{
  "vermeldingen": {
    "NaN": {
      "Europa": {
        "Polen": {
          "Concert": {
            "Klassieke muziek": 2
          }
        },
        "Verenigd Koninkrijk": {
          "Concert": {
            "Klassieke muziek": 2
          }
        },
        "Belgi\u00eb": {
          "Tentoonstelling": {
            "Installatiekunst": 1
          }
        }
      }
    },
    "Sociaal-cultureel en amateur": {
      "Afrika": {
        "Congo-Kinshasa": {
          "Concert": {
            "Thema onbepaald": 1
          }
        }
      }
    },
    "Rest": {
      "Noord-Amerika": {
        "Verenigde Staten": {
          "Concert": {
            "Folk en wereldmuziek": 1
          }
        }
      },
      "Europa": {
        "Belgi\u00eb": {
          "Concert": {
            "Folk en wereldmuziek": 1
          }
        }
      }
    },
    "Privaat": {
      "Oceani\u00eb": {
        "Australi\u00eb": {
          "Concert": {
            "Klassieke muziek": 4
   

We kunnen nu deze gegevens samenballen in een tabel.

In [51]:
lines = []
for fusiegemeente in data:
    lat = data[fusiegemeente]["latitude"]
    lon = data[fusiegemeente]["longitude"]
    prov = data[fusiegemeente]["provincie"]
    for organisatie in data[fusiegemeente]["vermeldingen"]:
        for continent in data[fusiegemeente]["vermeldingen"][organisatie]:
            for land in data[fusiegemeente]["vermeldingen"][organisatie][continent]:
                for discipline in data[fusiegemeente]["vermeldingen"][organisatie][continent][land]:
                    for subdiscipline in data[fusiegemeente]["vermeldingen"][organisatie][continent][land][discipline]:
                        lines.append([fusiegemeente, lat, lon, prov, continent, land, discipline, subdiscipline, organisatie,
                                      data[fusiegemeente]["vermeldingen"][organisatie][continent][land][discipline][subdiscipline]])
res = DataFrame(lines, columns=['Gemeente', 'Latitude', 'Longitude', 'Provincie', 'Continent', 'Land', 'Discipline', 'Subdiscipline', 'Typering', 'Frequentie'])

De tabel bevat dus de volgende informatie:

In [52]:
res.head()

,Gemeente,Latitude,Longitude,Provincie,Continent,Land,Discipline,Subdiscipline,Typering,Frequentie
0,Ruiselede,51.04131,3.397051,West Flanders,Europa,Polen,Concert,Klassieke muziek,NaN,2
1,Ruiselede,51.04131,3.397051,West Flanders,Europa,Verenigd Koninkrijk,Concert,Klassieke muziek,NaN,2
2,Ruiselede,51.04131,3.397051,West Flanders,Europa,België,Tentoonstelling,Installatiekunst,NaN,1
3,Ruiselede,51.04131,3.397051,West Flanders,Afrika,Congo-Kinshasa,Concert,Thema onbepaald,Sociaal-cultureel en amateur,1
4,Ruiselede,51.04131,3.397051,West Flanders,Noord-Amerika,Verenigde Staten,Concert,Folk en wereldmuziek,Rest,1


We schrijven tot slot deze tabel uit naar een CSV bestand voor verdere analyse.

In [53]:
res.to_csv("landsvermeldingen.csv")

## Paren van landsvermeldingen

Een andere interessante is om paartjes van landsvermeldingen te zien. Wordt Frankrijk vaak met Tunisie in 1 event aangehaald? Of Duitsland met Oostenrijk?

In [44]:
events_met_landen = []

count = 1
for row in df.iterrows():
    landenlijst = []
    tekst = row[1]["Tekst"]
    if count % 5000 == 0:
        print(count, "of", len(df.index))
    count += 1
    for land in set(landen["Land"].values):
        regex = compile(r"\b(" + r"|".join(landen[landen["Land"] == land]["Mention"]) + r")\b")
        amount_of_mentions = len(regex.findall(str(tekst)))
        if amount_of_mentions > 0:
            landenlijst.append(land)
    events_met_landen.append(landenlijst)

100 of 43450


KeyboardInterrupt: 

In [29]:
print(events_met_landen[0:10])

[['Vlaanderen'], [], ['Vlaanderen'], ['Vlaanderen'], [], [], ['Frankrijk'], ['Frankrijk'], [], [], ['België'], ['Polen', 'Japan'], ['België'], ['Duitsland', 'Rusland'], [], [], [], [], [], [], [], ['Vlaanderen'], [], [], ['Nederland'], [], [], [], ['Argentinië'], ['Bosnië en Herzegovina'], ['Vlaanderen', 'Wallonië'], ['Verenigde Staten', 'Estland'], [], [], [], [], [], ['Noorwegen'], [], [], [], [], [], [], [], [], ['Nederland', 'Verenigde Staten', 'Verenigd Koninkrijk', 'België'], [], ['Marokko'], ['China']]


In [40]:
pairs = []
landenset = set()
for landenlijst in events_met_landen:
    for land in landenlijst:
        landenset.add(land)
    for combi in combinations(landenlijst, 2):
        pairs.append(combi)
pairs_counter = Counter(pairs)
pairs_counter_mat = {}
for c in pairs_counter:
    if c[0] not in pairs_counter_mat:
        pairs_counter_mat[c[0]] = {}
    if c[1] not in pairs_counter_mat:
        pairs_counter_mat[c[1]] = {}
    pairs_counter_mat[c[0]][c[1]] = pairs_counter[c]
    pairs_counter_mat[c[1]][c[0]] = pairs_counter[c]
smat = {}
for land_a in landenset:
    for land_b in landenset:
        if land_a not in smat:
            smat[land_a] = {}
        if land_b not in smat:
            smat[land_b] = {}
        if land_a in pairs_counter_mat and land_b in pairs_counter_mat[land_a]:
            smat[land_a][land_b] = pairs_counter_mat[land_a][land_b]
            smat[land_b][land_a] = pairs_counter_mat[land_a][land_b]
        else:
            smat[land_a][land_b] = 0
            smat[land_b][land_a] = 0
DataFrame(smat)

,Argentinië,België,Bosnië en Herzegovina,China,Duitsland,Estland,Frankrijk,Japan,Marokko,Nederland,Noorwegen,Polen,Rusland,Verenigd Koninkrijk,Verenigde Staten,Vlaanderen,Wallonië
Argentinië,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
België,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
Bosnië en Herzegovina,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
China,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Duitsland,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
Estland,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
Frankrijk,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Japan,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Marokko,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Nederland,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
